In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import json
import matplotlib.pyplot as plt

from tqdm import tqdm
from collections import defaultdict

In [2]:
from matplotlib import rc
import matplotlib.font_manager as fm
import matplotlib

# 한글깨짐
fm.get_fontconfig_fonts()
font_location = '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf'

font_name = fm.FontProperties(fname=font_location).get_name()
matplotlib.rc('font', family=font_name)

/tmp/ipykernel_3114/703292248.py:6: MatplotlibDeprecationWarning: 
The get_fontconfig_fonts function was deprecated in Matplotlib 3.5 and will be removed two minor releases later.
  fm.get_fontconfig_fonts()


In [3]:
"""
    총 aspect : 25개
    제품 전체 : 품질, 디자인, 편의성, 일반, 가격, 다양성, 인지도 (7개)
    패키지/구성품 : 디자인, 다양성, 일반, 편의성, 품질, 가격 (6개)
    본품 : 일반, 다양성, 품질, 인지도, 편의성, 디자인, 가격 (7개)
    브랜드 : 일반, 인지도, 디자인, 품질, 가격 (5개)

    sentiment : negative, netural, positive (3개)
"""

'\n    총 aspect : 25개\n    제품 전체 : 품질, 디자인, 편의성, 일반, 가격, 다양성, 인지도 (7개)\n    패키지/구성품 : 디자인, 다양성, 일반, 편의성, 품질, 가격 (6개)\n    본품 : 일반, 다양성, 품질, 인지도, 편의성, 디자인, 가격 (7개)\n    브랜드 : 일반, 인지도, 디자인, 품질, 가격 (5개)\n\n    sentiment : negative, netural, positive (3개)\n'

In [32]:
TOTAL_ASPECT= ['본품#품질', '제품 전체#일반', '본품#일반', '제품 전체#가격', '제품 전체#편의성', '제품 전체#디자인',
       '패키지/구성품#일반', '패키지/구성품#편의성', '제품 전체#품질', '브랜드#가격', '본품#편의성',
       '본품#디자인', '브랜드#일반', '브랜드#품질', '제품 전체#인지도', '패키지/구성품#디자인', '본품#다양성',
       '패키지/구성품#품질', '패키지/구성품#다양성', '브랜드#인지도', '본품#가격', '본품#인지도']

# 본품#품질', '제품 전체#일반' 제외
STAGE2_ASPECT= ['본품#일반', '제품 전체#가격', '제품 전체#편의성', '제품 전체#디자인',
       '패키지/구성품#일반', '패키지/구성품#편의성', '제품 전체#품질', '브랜드#가격', '본품#편의성',
       '본품#디자인', '브랜드#일반', '브랜드#품질', '제품 전체#인지도', '패키지/구성품#디자인', '본품#다양성',
       '패키지/구성품#품질', '패키지/구성품#다양성', '브랜드#인지도', '본품#가격', '본품#인지도', '패키지/구성품#가격']
# train에는 패키지/구성품#가격 없는데 dev에는 있는..


SENTIMENT= ['negative', 'positive', 'neutral']


stage1_aspect_dict= {'본품#품질': 0, '제품 전체#일반': 1, 'OTHERS': 2}

stage2_num_to_aspect_dict= {idx:value for idx, value in enumerate(STAGE2_ASPECT)}
stage2_aspect_to_num_dict= {value:idx for idx, value in enumerate(STAGE2_ASPECT)}
print(stage2_aspect_to_num_dict)

stage1_sentiment_dict= {'positive': 0, 'OTHERS': 1}
stage2_sentiment_dict= {'negative': 0, 'neutral': 1}






{'본품#일반': 0, '제품 전체#가격': 1, '제품 전체#편의성': 2, '제품 전체#디자인': 3, '패키지/구성품#일반': 4, '패키지/구성품#편의성': 5, '제품 전체#품질': 6, '브랜드#가격': 7, '본품#편의성': 8, '본품#디자인': 9, '브랜드#일반': 10, '브랜드#품질': 11, '제품 전체#인지도': 12, '패키지/구성품#디자인': 13, '본품#다양성': 14, '패키지/구성품#품질': 15, '패키지/구성품#다양성': 16, '브랜드#인지도': 17, '본품#가격': 18, '본품#인지도': 19, '패키지/구성품#가격': 20}


In [25]:
TRAIN_DATA_PATH= './data/nikluge-sa-2022-train.jsonl'
DEV_DATA_PATH= './data/nikluge-sa-2022-dev.jsonl'
TEST_DATA_PATH= './data/nikluge-sa-2022-test.jsonl'

In [26]:
def read_jsonl(path):
    data= []
    with open(path, 'r') as f:
        for i, line in tqdm(enumerate(f)):
            data.append(json.loads(line))
    
    return data

In [27]:
trainset= read_jsonl(TRAIN_DATA_PATH)
devset= read_jsonl(DEV_DATA_PATH)
testset= read_jsonl(TEST_DATA_PATH)

3001it [00:00, 135909.33it/s]
2794it [00:00, 145244.23it/s]
2127it [00:00, 179538.83it/s]


In [33]:
def convert_jsonl_to_df(jsonl_data):
    multi_label_count_dict= defaultdict(lambda: 0)
    data_dict= defaultdict(list)
    for js_data in tqdm(jsonl_data):
        id, sentence, annotation= js_data['id'], js_data['sentence_form'], js_data['annotation']
        
        for annot in annotation:
            tmp_aspect, _, sentiment= annot
            aspect_1, aspect_2= tmp_aspect.split('#')
            
            data_dict['id'].append(id)
            data_dict['sentence'].append(sentence)
            data_dict['aspect_1'].append(aspect_1)
            data_dict['aspect_2'].append(aspect_2)
            data_dict['aspect_concat'].append(tmp_aspect)
            data_dict['sentiment'].append(sentiment)

            # for aspect stage1
            label_aspect_1= -1
            if tmp_aspect == '본품#품질':
                label_aspect_1= 0
            elif tmp_aspect == '제품 전체#일반':
                label_aspect_1= 1
            else:
                label_aspect_1= 2
            data_dict['label_aspect_1'].append(label_aspect_1)

            # for aspect stage2
            label_aspect_2= -1
            if label_aspect_1== 2:
                label_aspect_2= stage2_aspect_to_num_dict[tmp_aspect]
            # 본품#품질 / 제품 전체#일은 skip
            data_dict['label_aspect_2'].append(label_aspect_2)

            # for sentiment stage1
            label_sentiment_1= -1
            if sentiment== 'positive':
                label_sentiment_1= 0
            else:
                label_sentiment_1= 1
            data_dict['label_sentiment_1'].append(label_sentiment_1)
            
            # for sentiment stage2
            label_sentiment_2= -1
            if label_sentiment_1== 1:
                label_sentiment_2= stage2_sentiment_dict[sentiment]
            data_dict['label_sentiment_2'].append(label_sentiment_2)
                
    return pd.DataFrame(data_dict)



In [39]:
train_df= convert_jsonl_to_df(trainset) 
dev_df= convert_jsonl_to_df(devset)
dev_df.head()

100%|██████████| 2794/2794 [00:00<00:00, 444099.04it/s]


,id,sentence,aspect_1,aspect_2,aspect_concat,sentiment,label_aspect_1,label_aspect_2,label_sentiment_1,label_sentiment_2
0,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품,편의성,본품#편의성,positive,2,8,0,-1
1,nikluge-sa-2022-dev-00002,목욕할 때마다 넣어봤는데(샤워는 자주 해도 목욕은 그렇게 자주가 아님.. 이것도 약...,본품,일반,본품#일반,positive,2,0,0,-1
2,nikluge-sa-2022-dev-00003,혹시 오래된 거 팔지 않나 고민했었는데 쑥향기 자체가 페퍼민트처럼 신선하고 포장도 ...,본품,일반,본품#일반,positive,2,0,0,-1
3,nikluge-sa-2022-dev-00003,혹시 오래된 거 팔지 않나 고민했었는데 쑥향기 자체가 페퍼민트처럼 신선하고 포장도 ...,패키지/구성품,디자인,패키지/구성품#디자인,positive,2,13,0,-1
4,nikluge-sa-2022-dev-00004,전에는 목욕할 때 입욕제 넣고 하는 거 좋아하지 않았는데 한약재로 바꿔보니 약용까지...,제품 전체,품질,제품 전체#품질,positive,2,6,0,-1


In [38]:
train_df.groupby('label_sentiment_2').count()

,id,sentence,aspect_1,aspect_2,aspect_concat,sentiment,label_aspect_1,label_aspect_2,label_sentiment_1
label_sentiment_2,,,,,,,,,
-1,3102,3102,3102,3102,3102,3102,3102,3102,3102
0,58,58,58,58,58,58,58,58,58
1,95,95,95,95,95,95,95,95,95


In [31]:
train_df['aspect_concat'].unique()
train_df['sentiment'].unique()

array(['negative', 'positive', 'neutral'], dtype=object)

In [40]:
train_df.to_csv('./data/process_train.csv', index= False)
dev_df.to_csv('./data/process_dev.csv', index= False)